In [14]:
import pandas as pd
import polars as pl
import numpy as np
import scipy.stats as ss
import yfinance as yf
import ipywidgets as widgets
from IPython.display import display

In this example, I create a simple portfolio made of 2 largely diversified stock and bond indexes with a long time series:
1) IEF: iShares 7-10 Year Treasury Bond ETF (USD)
2) ACWI: The MSCI ACWI captures large and mid cap representation across 23 Developed Markets (DM) and 24 Emerging Markets (EM) countries (USD).


In [2]:
# obtain the price time series from Yahoo Finance
start="1990-05-01"

tickers=["IEF","ACWI"]
tickers_dl=["IEF","SPY"] #"SPY" is taken just to keep the date column, otherwise if does not show it
df=yf.download(tickers_dl,start=start,interval="1mo")['Adj Close']
df=pl.from_pandas(df,include_index=True).drop_nulls()
df=df.cast({"Date":pl.Date})
df

[*********************100%%**********************]  2 of 2 completed


Date,IEF,SPY
date,f64,f64
2002-08-01,45.245888,60.831509
2002-09-01,46.828938,54.210171
2002-10-01,46.265507,58.933765
2002-11-01,45.277279,62.568886
2002-12-01,46.52916,58.740719
2003-01-01,46.6842,57.577576
2003-02-01,47.557934,56.801483
2003-03-01,47.400772,56.694416
2003-04-01,47.460377,61.739368


In [3]:
# import from CSV - Let's borrow ACWI from Prof PC    
df2=pl.read_csv("https://raw.githubusercontent.com/paolocole/Stock-Indexes-Historical-Data/main/GROSS/USD/LARGE_AND_MID_CAP/REGIONS/MSCI%20ACWI.csv")
    #df=df.cast({"Date":pl.Date})
df2=df2.with_columns((pl.col("Date")+"-01").alias("Date"))
date2=df2["Date"].str.to_date("%Y-%m-%d")
df2=df2.with_columns(date2)



In [4]:
# merge the two sources - let's keep dates available in both time series
df2=df2.join(df,on="Date",how='inner')
df2=df2.drop("SPY")
df=df2
print(df.head(2))
print(df.tail(2))

shape: (2, 3)
┌────────────┬────────────┬───────────┐
│ Date       ┆ ACWI       ┆ IEF       │
│ ---        ┆ ---        ┆ ---       │
│ date       ┆ f64        ┆ f64       │
╞════════════╪════════════╪═══════════╡
│ 2002-08-01 ┆ 269.550083 ┆ 45.245888 │
│ 2002-09-01 ┆ 239.985118 ┆ 46.828938 │
└────────────┴────────────┴───────────┘
shape: (2, 3)
┌────────────┬─────────────┬───────────┐
│ Date       ┆ ACWI        ┆ IEF       │
│ ---        ┆ ---         ┆ ---       │
│ date       ┆ f64         ┆ f64       │
╞════════════╪═════════════╪═══════════╡
│ 2023-06-01 ┆ 1539.573475 ┆ 93.292084 │
│ 2023-07-01 ┆ 1596.391861 ┆ 92.680801 │
└────────────┴─────────────┴───────────┘


In [5]:
price_norm=df.clone()
price_change=df.clone()
for i in range(1,len(tickers)+1):
    ticker = tickers[i-1]
    first_price=price_norm.item(0,ticker)
    print(ticker,first_price)
    price_norm=price_norm.with_columns((100*pl.col(ticker)/first_price).alias(ticker)) # normalize at 100 at time series start
    price_change=price_change.with_columns(pl.col(ticker).pct_change().alias(ticker+"_change"))
price_change=price_change.drop(tickers).drop_nulls()

IEF 45.245887756347656
ACWI 269.5500833006467


In [6]:
price_norm.head()

Date,ACWI,IEF
date,f64,f64
2002-08-01,100.0,100.0
2002-09-01,89.031736,103.498771
2002-10-01,95.587893,102.253506
2002-11-01,100.8167,100.069379
2002-12-01,96.01011,102.836218


In [7]:
price_change.head(10)

Date,IEF_change,ACWI_change
date,f64,f64
2002-09-01,0.034988,-0.109683
2002-10-01,-0.012032,0.073638
2002-11-01,-0.02136,0.054702
2002-12-01,0.027649,-0.047677
2003-01-01,0.003332,-0.029187
2003-02-01,0.018716,-0.017524
2003-03-01,-0.003305,-0.00378
2003-04-01,0.001257,0.089302
2003-05-01,0.036783,0.058196


In [8]:
price_norm.plot(x="Date")

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [9]:
pchange_descr=price_change.describe().drop("Date")
pchange_descr

statistic,IEF_change,ACWI_change
str,f64,f64
"""count""",251.0,251.0
"""null_count""",0.0,0.0
"""mean""",0.003044,0.008158
"""std""",0.019236,0.045419
"""min""",-0.054739,-0.197906
"""25%""",-0.008667,-0.015411
"""50%""",0.002025,0.013507
"""75%""",0.013381,0.035108
"""max""",0.077309,0.123633


In [10]:
price_change.plot.box()

:BoxWhisker   [Variable]   (value)

In [11]:
#calculate spearman rank correlation (currently easier with Pandas) and pearson correlation 
price_corr=price_change.drop("Date")
price_corr=price_corr.to_pandas()
price_corr1=price_corr.corr(method='spearman')
print('spearman\n',price_corr1)
price_corr2=price_corr.corr(method='pearson')
print('\npearson\n',price_corr2)

price_corr1=pl.from_pandas(price_corr1)

spearman
              IEF_change  ACWI_change
IEF_change     1.000000    -0.159876
ACWI_change   -0.159876     1.000000

pearson
              IEF_change  ACWI_change
IEF_change      1.00000     -0.15781
ACWI_change    -0.15781      1.00000


In [12]:
# verify autocorrelation for any weird surprises
lags = [1]
lags2 = list(range(3,61,3))
lags = lags+lags2
time_series=price_change.to_numpy()
time_series=np.delete(time_series,0,1)
corr=np.zeros([len(lags),len(tickers)+1])
for i in range(len(tickers)):
    ts = time_series[:,i]
    ts = np.reshape(ts,ts.size)
    count=0
    for l in lags:
        arr1=ts[:-l]
        arr2=ts[l:]
        correl = np.corrcoef(arr1,arr2)[0][1]
        corr[count,0]=l
        corr[count,i+1]=correl
        count+=1
column_names=["lag (months)"]+tickers
corr_dict={}
for i in range(len(column_names)):
    corr_dict[column_names[i]]=corr[:,i].tolist()
corr_df=pl.DataFrame(corr_dict)
corr_df.plot.bar(x="lag (months)",y=tickers,title="Autocorrelation",ylim=[-1,1],subplots=False).opts(xrotation=90)

:Bars   [lag (months),Variable]   (value)

In [17]:
# Anderson-Darling normality test - it should confirm that distribution of returns is not normal
dist = price_change.to_numpy()
dist = np.delete(dist,0,1)
for t in range(len(tickers)):
    norm_fit = ss.anderson(dist[:,t],'norm')
    test_stat = norm_fit.statistic
    critical_5p = norm_fit.critical_values[2]

    if test_stat > critical_5p:
        print(tickers[t],": Distribution is NOT normal with significance of 5%")
    else:
        print(tickers[t],": Distribution is normal with significance of 5%")

IEF : Distribution is NOT normal with significance of 5%
ACWI : Distribution is NOT normal with significance of 5%


In [18]:
# print output of monthly returns
price_change.write_excel("TimeSeries_output.xlsx",worksheet="Market")